In [172]:
import pandas as pd

In [173]:

df = pd.read_csv("space/imputed2.csv")

pre_remov = df.shape[1]
df = df.loc[:, df.nunique() != 1]
print(f"Removed {pre_remov - df.shape[1]} columns with only one unique value")

Removed 0 columns with only one unique value


In [174]:
mean_wl_time = df['wl_time'].mean()
print(f"Mean waiting list time: {mean_wl_time}")

df['wl_time'] = df['wl_time'].apply(lambda x: 0 if x < mean_wl_time else 1)
print(df['wl_time'].value_counts(normalize=True))


Mean waiting list time: 192.52839426430694
0    0.716807
1    0.283193
Name: wl_time, dtype: float64


In [175]:
import statsmodels.api as sm
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
CV = 5

X = df.drop(['wl_time'], axis=1)
#X = X.iloc[:, :20]# Scale down X to only 10 features
y = df['wl_time']

scaler = StandardScaler()
X = scaler.fit_transform(X)

log = LogisticRegression(max_iter=1000)
scores = cross_val_score(log, X, y, cv=CV, scoring='f1').mean()
print("F1 score:", scores)


logit_model = sm.OLS(y, X)
result = logit_model.fit()
print(result.summary())

X = sm.add_constant(X)
X_opt = df

print(X_opt.head())
SIGNIFICANCE_LEVEL = 0.05

while True:
    print(f"Number of variables: {len(X_opt.columns)}")
    model = sm.OLS(y, X_opt).fit()
    p_values = model.pvalues
    max_p_value = p_values.max()

    if max_p_value <= SIGNIFICANCE_LEVEL:
        break

    print(f"Removing {p_values.idxmax()} with p-value {max_p_value}")
    X_opt = X_opt.drop([p_values.idxmax()], axis=1)

X_opt.head()

#result = logit_model.fit()
#print(result.summary())

log = LogisticRegression(max_iter=1000)
scores = cross_val_score(log, X_opt, y, cv=CV, scoring='f1').mean()
print("F1 score:", scores)


F1 score: 0.8952750557996195
                                 OLS Regression Results                                
Dep. Variable:                wl_time   R-squared (uncentered):                   0.359
Model:                            OLS   Adj. R-squared (uncentered):              0.358
Method:                 Least Squares   F-statistic:                              323.0
Date:                Tue, 18 Apr 2023   Prob (F-statistic):                        0.00
Time:                        20:12:56   Log-Likelihood:                         -43793.
No. Observations:               77410   AIC:                                  8.785e+04
Df Residuals:                   77276   BIC:                                  8.909e+04
Df Model:                         134                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
------------

In [176]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm

# Backward elimination using F1 score as criterion

CV = 5

X = df.drop(['wl_time'], axis=1)
X = X.iloc[:, :20]# Scale down X to only 10 features

y = df['wl_time']

scaler = StandardScaler()
X = scaler.fit_transform(X)

log = LogisticRegression(max_iter=1000)

features = X.shape[1]
removed_features = []

tracker = tqdm(total=features, desc="Iterations")

for iteration in range(features):
    f1_iteration_baseline = cross_val_score(log, X, y, cv=CV, scoring="f1").mean()
    #f1_iteration_baseline = log.fit(X, y).score(X, y)
    print(f"{iteration}/{features}", f"Baseline F1 score: {f1_iteration_baseline}")

    f1_results = {}
    eligible_features = [i for i in range(features) if i not in removed_features]

    tracker2 = tqdm(total=features-iteration, desc="Features")


    for feature in eligible_features:
        X_subset = X[:, [i for i in range(features) if i not in removed_features + [feature]]]

        f1 = cross_val_score(log, X_subset, y, cv=CV, scoring="f1").mean()
        #f1 = log.fit(X_subset, y).score(X_subset, y)

        f1_results[feature] = f1_iteration_baseline - f1
        print(f"{iteration}/{features}",
              f"{feature}/{len(eligible_features)}",
              f"F1 = {f1}"
              )
        tracker2.update(1)


        # 90 91 = -1 # BAD
        # 90 89 = 1 # GOOD
        # 90 90 = 0 # BAD

    worst_feature = min(f1_results, key=f1_results.get)
    best_feature = max(f1_results, key=f1_results.get)

    if worst_feature == best_feature:
        print("No feature to remove")
        break

    print(f"{iteration}/{features}",
          f"Best feature: {best_feature} F1 = {f1_results[best_feature]}",
          f"Worst feature: {worst_feature} F1 = {f1_results[worst_feature]}",
          )

    removed_features.append(worst_feature)
    tracker.update(1)

print("ranking:", removed_features)



Iterations:   0%|          | 0/20 [00:00<?, ?it/s]

0/20 Baseline F1 score: 0.3607947736787299


Features:   0%|          | 0/20 [00:00<?, ?it/s]

0/20 0/20 F1 = 0.36052876718467175
0/20 1/20 F1 = 0.3495625775774318
0/20 2/20 F1 = 0.3559559075333286
0/20 3/20 F1 = 0.35983261752095164
0/20 4/20 F1 = 0.36053856744595564
0/20 5/20 F1 = 0.35337475872728275
0/20 6/20 F1 = 0.3614389043544609
0/20 7/20 F1 = 0.35909242007446973
0/20 8/20 F1 = 0.3346404537997382
0/20 9/20 F1 = 0.3226628649354059
0/20 10/20 F1 = 0.36040109513966023
0/20 11/20 F1 = 0.36080327370300314
0/20 12/20 F1 = 0.3603882382241522
0/20 13/20 F1 = 0.35973271948027435
0/20 14/20 F1 = 0.35550269583625266
0/20 15/20 F1 = 0.3605916608849584
0/20 16/20 F1 = 0.2996030110122348
0/20 17/20 F1 = 0.34695054716962487
0/20 18/20 F1 = 0.29257285668340194
0/20 19/20 F1 = 0.35856589929878274
0/20 Best feature: 18 F1 = 0.06822191699532798 Worst feature: 6 F1 = -0.0006441306757309917
1/20 Baseline F1 score: 0.3607947736787299


Features:   0%|          | 0/20 [00:00<?, ?it/s]

...

17/20 5/3 F1 = 0.11842262031590867
17/20 9/3 F1 = 0.0810325607042371
17/20 18/3 F1 = 0.13565196498908993
17/20 Best feature: 9 F1 = 0.27976221297449283 Worst feature: 18 F1 = 0.22514280868963998
18/20 Baseline F1 score: 0.3607947736787299


Features:   0%|          | 0/20 [00:00<?, ?it/s]

18/20 5/2 F1 = 0.047141271727279
18/20 9/2 F1 = 0.0
18/20 Best feature: 9 F1 = 0.3607947736787299 Worst feature: 5 F1 = 0.3136535019514509
19/20 Baseline F1 score: 0.3607947736787299


Features:   0%|          | 0/20 [00:00<?, ?it/s]

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/edap01/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/anaconda3/envs/edap01/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 1138, in fit
    X, y = self._validate_data(
  File "/opt/homebrew/anaconda3/envs/edap01/lib/python3.10/site-packages/sklearn/base.py", line 596, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/opt/homebrew/anaconda3/envs/edap01/lib/python3.10/site-packages/sklearn/utils/validation.py", line 1074, in check_X_y
    X = c

In [ ]:
"""
import numpy as np
from time import time
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

X = df.drop(['wl_time'], axis=1)
y = df['wl_time']

scaler = StandardScaler()
X = scaler.fit_transform(X)

log = LogisticRegression(max_iter=1000)

# Evaluate the model using all features and the F1 score metric
print("Evaluating model with all features")
scores = cross_val_score(log, X, y, cv=5, scoring='f1_macro')
avg_score = np.mean(scores)
print("Initial F1 score:", avg_score)

# Iterate over all features and remove the one that has the smallest impact on the average F1 score
n_features = X.shape[1]
removed_features = []
t_total_start = time()

for i in range(n_features):
    t_start = time()
    print("Evaluating feature", i, "of", n_features)
    # Compute the F1 score for the current set of features
    current_features = [j for j in range(n_features) if j not in removed_features + [i]]
    X_subset = X[:, current_features]
    scores_subset = cross_val_score(log, X_subset, y, cv=5, scoring='f1_macro')
    avg_score_subset = np.mean(scores_subset)

    # Compute the difference in F1 score between the current set of features and the original set of features
    score_diff = avg_score - avg_score_subset

    # Remove the feature that has the smallest impact on the F1 score
    if score_diff > 0:
        removed_features.append(i)
        avg_score = avg_score_subset
        print("Removed feature", i, "F1 score:", avg_score)
    t_stop = time()
    t_elapsed = t_stop - t_start
    t_left = (n_features - i - 1) * t_elapsed
    print("Time elapsed:", t_elapsed, "Time left:", t_left)
    """


In [ ]:
#Print removed features
print("Removed features:", removed_features)